In [20]:
import numpy as np
import torch
import tensorflow as tf
import scipy.stats as stats

Numpy Implementation

In [21]:
def blackScholes_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    Phi = stats.norm.cdf
    d_1 = (np.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))
    d_2 = d_1 - sigma*np.sqrt(dt)
    return S*Phi(d_1) - K*np.exp(-r*dt)*Phi(d_2)

In [30]:
#%%timeit
S_0 = 100
K = 101
dt = 1
sigma = 7
r = 0.07

npv_numpy= blackScholes_py(S_0, K, dt, sigma, r)

In [31]:
print(npv_numpy)

99.95485181284957


In [71]:
ef blackScholes_tf_pricer():
    # Build the static computational graph
    S = tf.placeholder(tf.float32)
    K = tf.placeholder(tf.float32)
    dt = tf.placeholder(tf.float32)
    sigma = tf.placeholder(tf.float32)
    r = tf.placeholder(tf.float32)
    Phi = tf.distributions.Normal(0.,1.).cdf
    d_1 = (tf.log(S / K) + (r+sigma**2/2)*dt) / (sigma*tf.sqrt(dt))
    d_2 = d_1 - sigma*tf.sqrt(dt)
    npv =  S*Phi(d_1) - K*tf.exp(-r*dt)*Phi(d_2)
    greeks = tf.gradients(npv, [S, sigma, r, K, dt])
    def execute_graph(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
        with tf.Session() as sess:
            res = sess.run([npv, greeks], 
                           {
                               S: S_0,
                               K : strike,
                               r : riskfree_rate,
                               sigma: implied_vol,
                               dt: time_to_expiry})
        return res
    return execute_graph

SyntaxError: invalid syntax (<ipython-input-71-d5bff07bd5e3>, line 1)

In [68]:
def blackScholes_pyTorch(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    Phi = torch.distributions.Normal(0,1).cdf
    d_1 = (torch.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*torch.sqrt(dt))
    d_2 = d_1 - sigma*torch.sqrt(dt)
    return S*Phi(d_1) - K*torch.exp(-r*dt)*Phi(d_2)

In [69]:
#%%timeit
S_0 = torch.tensor([8400.],requires_grad=True)
K = torch.tensor([8600.],requires_grad=True)
T = torch.tensor([0.08],requires_grad=True)
sigma = torch.tensor([0.18],requires_grad=True)
r = torch.tensor([0.07],requires_grad=True)
npv_pytorch = blackScholes_pyTorch(S_0, K, T, sigma, r)

In [70]:
npv_pytorch

tensor([106.7126], grad_fn=<SubBackward0>)

##Greeks

In [52]:
#%%timeit
S_0 = torch.tensor([100.],requires_grad=True)
K = torch.tensor([101.],requires_grad=True)
T = torch.tensor([1.],requires_grad=True)
sigma = torch.tensor([0.3],requires_grad=True)
r = torch.tensor([0.01],requires_grad=True)
npv_pytorch = blackScholes_pyTorch(S_0, K, T, sigma, r)
npv_pytorch.backward()
delta_pytorch = S_0.grad
vega_pytorch = sigma.grad
rho_pytorch = r.grad
theta_pytorch = T.grad
digital_pytoch = -K.grad
#gamma = torch.autograd.grad(npv_pytorch, S_0, create_graph=True )


In [53]:
print("Delta =",float(delta_pytorch))
print("Vega =",float(vega_pytorch))
print("Rho =",float(rho_pytorch))
print("Theta =",float(theta_pytorch))

Delta = 0.5596830248832703
Vega = 39.446956634521484
Rho = 44.042572021484375
Theta = 6.35746955871582


In [ ]:
##Monte Carlo simulation

In [54]:
def monte_carlo_down_out_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, steps, samples):
    stdnorm_random_variates = np.random.randn(samples, steps)
    S = S_0
    K = strike
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    sigma = implied_vol
    r = riskfree_rate
    B = barrier
    
    B_shift = B*np.exp(0.5826*sigma*np.sqrt(dt))
    S_T = S * np.cumprod(np.exp((r-sigma**2/2)*dt+sigma*np.sqrt(dt)*stdnorm_random_variates), axis=1)
    non_touch = (np.min(S_T, axis=1) > B_shift)*1
    call_payout = np.maximum(S_T[:,-1] - K, 0)
    npv = np.mean(non_touch * call_payout)
    return np.exp(-time_to_expiry*r)*npv

In [55]:
monte_carlo_down_out_py(100., 110., 2., 0.2, 0.03, 90., 1000, 100000)

7.145556483178348

In [ ]:
##Exotic options /Barrier Option

In [60]:
def monte_carlo_down_out_torch(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, steps, samples):
    stdnorm_random_variates = torch.distributions.Normal(0,1).sample((samples, steps))
    S = S_0
    K = strike
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    sigma = implied_vol
    r = riskfree_rate
    B = barrier
    
    B_shift = B*torch.exp(0.5826*sigma*torch.sqrt(dt))
    S_T = S * torch.cumprod(torch.exp((r-sigma**2/2)*dt+sigma*torch.sqrt(dt)*stdnorm_random_variates), dim=1)
    non_touch = torch.min(S_T, dim=1)[0] > B_shift
    call_payout = S_T[:,-1] - K
    call_payout[call_payout<0]=0
    npv = torch.mean(non_touch.type(torch.FloatTensor) * call_payout)
    return torch.exp(-time_to_expiry*r)*npv

In [64]:
S = torch.tensor([8400.],requires_grad=True)
K = torch.tensor([8600.],requires_grad=True)
T = torch.tensor([0.08],requires_grad=True)
sigma = torch.tensor([0.18],requires_grad=True)
r = torch.tensor([0.07],requires_grad=True)
B = torch.tensor([90.],requires_grad=True)
npv_torch_mc = monte_carlo_down_out_torch(S, K, T, sigma, r, B, 1000, 100000)
npv_torch_mc.backward()

In [65]:
npv_torch_mc

tensor([106.6766], grad_fn=<MulBackward0>)